# Hatchet Query Language

This notebook explores [Hatchet](https://github.com/LLNL/hatchet) queries from its [**Call Path Query Language**](https://hatchet.readthedocs.io/en/latest/query_lang.html), specifically **Category 8: Predicate Combination through Other Operations**. The notebook covers different query types that are capable of finding nodes from [GraphFrame](https://hatchet.readthedocs.io/en/latest/user_guide.html) objects matching an exclusive disjunction (XOR) of two or more query node predicates.

In [ ]:
# display documentation for Hatchet GraphFrame
from IPython.display import Markdown, display
display(Markdown("../common/documentation/hatchet-query-language.md"))

In [2]:
# display documentation for object-based dialect
display(Markdown("../common/documentation/base-query-language-08.md"))

The **Query Language** finds all paths in a call graph that match properties described by the query applied to profiling data. It enables Hatchet’s Jupyter notebook-based interactive visualization to provide users with a simple and intuitive way to massively reduce the profiling data interactively. The **Query Language** has two dialects (Object-based Dialect and String-based Dialect), that simplify its use under diverse circumstances. 

## Category 8: Predicate Combination through Other Operations (e.g., XOR)

Category 8 expands on combining query predicates through exclusive disjunction or exclusive OR with the `XOR` logical operator.The Hatchet base Query Language allows us to use more complex logic than AND, OR, and NOT in query node predicates. A complex logic we explore in this notebook is `XOR`.



In [3]:
# display dataset information 
display(Markdown("../common/documentation/dataset-information-with-regex.md"))

### Loading profile data as Hatchet GraphFrame

Hatchet queries are only defined on Hatchet GraphFrames. 
Obtaining a hatchet GraphFrame is straight forward:

1. Import hatchet
2. Import [python regular expression operations](https://docs.python.org/3/library/re.html) 
3. Use the appropriate reader for the profile/trace at hand

We first load a [Caliper](https://github.com/LLNL/Caliper) profile in JSON format, where Caliper is a performance profiling library developed by the Lawrence Livermore National Lab (LLNL).

This example profile is profiled from [LULESH (Livermore Unstructured Lagrangian Explicit Shock Hydrodynamics)](https://asc.llnl.gov/codes/proxy-apps/lulesh), a performance report data generated by Caliper. LULESH is a highly simplified application designed to solve the Sedov Blast problem, which is a standard hydrodynamics test problem. It performs a hydrodynamics stencil calculation using both MPI and OpenMP to achieve parallelism. 

This is an interesting profile because it covers a relatively large number of nodes (45 nodes) and spends considerable time in MPI communication routines.


In [ ]:
import hatchet as ht
import re
gf = ht.GraphFrame.from_caliper("../../data/lulesh-16nodes/lulesh-annotation-profile-512cores.json")

In [5]:
# display GraphFrame information 
display(Markdown("../common/documentation/graph-tree-information.md"))

### Displaying a Hatchet GraphFrame
A compact overview of a hatchet GraphFrame can be obtained using the `gf.tree()` function. We use this throughout the notebook to display the differences between an original GraphFrame and the resulting GraphFrame after applying a query.

In [ ]:
print(gf.tree())

In [7]:
# display DataFrame information 
display(Markdown("../common/documentation/dataframe-information.md"))

### Displaying a DataFrame
An additional detail perspective can be obtained by viewing the underlying data using a **DataFrame**. A Hatchet **DataFrame** holds all the numerical and categorical data associated with each node. 

In [ ]:
gf.dataframe

In [9]:
# why use drop index levels
display(Markdown("../common/documentation/drop-index-information.md"))

### Dropping index levels

As a precursor to defining queries, we drop the index level of the GraphFrame using the `drop_index_levels()` Hatchet function. Hatchet hierarchical indexing can be of two types, depending on whether there is a single metric per node or multiple set of metrics per node.  

If a node contains a single metric, the DataFrame will use an `Index` object containing the node column. If a node has an additional level of information, Hatchet creates a `MultiIndex` to store the information pertaining to multiple sets of metrics per node. `MultiIndex` stores the node column as the "top" level of the index, followed by additional information on the levels below. 

Based on the types of indexing (`Index or MultiIndex`), retrieving data from a DataFrame corresponding to a particular node either retrieves a single or multiple rows. This difference can cause issues when applying query node predicates.
Therefore, it is necessary to get rid of all index levels besides the node column through an aggregation operation on the GraphFrame. Then, a query node predicate can be applied to the GraphFrame. 

In [ ]:
gf.drop_index_levels()

In [16]:
# display query type 1 documentation
display(Markdown("../common/documentation/predicate-combination-other-operations.md"))

### Use more complex logic than AND, OR, and NOT in query node predicates 

This query type allows a user to search for nodes that meet the requirements of a complex logical operator, specifically `XOR` for this example. The predicates are combined using the XOR logical operator, which means that `only one` of the two predicates must be satisfied by a node. One can apply an XOR logic to predicates by using the following combination of AND, OR, and NOT logical operators:

(predicate 1 **AND** **NOT** predicate 2) **OR** (**NOT** predicate 1 **AND** predicate 2)

The following query matches all nodes where either the `name` metric `starts with MPI_` **AND** the `time` metric `is NOT less than 10000` **OR** the `name` metric does `NOT start with MPI_` **AND** the `time` metric `is less than 10000`. In this way, we accomplish the application of an `XOR` logical operation on the query node predicates such that:

* the `name` metric `starts with MPI_` **XOR** the `time` metric `is less than 10000`

In [ ]:
query_1 = ht.QueryMatcher().match("*", lambda row: (re.match("P?MPI_.*", row["name"]) is not None and not row["time"] < 10000) or (not re.match("P?MPI_.*", row["name"]) is not None and row["time"] < 10000))

The above query is passed to Hatchet’s `filter()` function to filter the input GraphFrame. The `filter()` function takes a user-supplied function or query object and applies that to all rows in the DataFrame. The resulting Series or DataFrame is used to filter the DataFrame to only return rows that are true.

In [ ]:
gf_filt = gf.filter(query_1)

The resulting GraphFrame now only lists the  nodes that matched the query:

In [ ]:
print(gf_filt.tree())

The query match is also reflected in the DataFrame:

In [ ]:
gf_filt.dataframe